# Poetry

We use poetry for dependency managaement

In [1]:
!curl -sSL https://install.python-poetry.org | python3 -

Retrieving Poetry metadata

# Welcome to Poetry!

This will download and install the latest version of Poetry,
a dependency and package manager for Python.

It will add the `poetry` command to Poetry's bin directory, located at:

/root/.local/bin

You can uninstall at any time by executing this script with the --uninstall option,
and these changes will be reverted.

Installing Poetry (2.1.1)
Installing Poetry (2.1.1): Creating environment
Installing Poetry (2.1.1): Installing Poetry
Installing Poetry (2.1.1): Creating script
Installing Poetry (2.1.1): Done

Poetry (2.1.1) is installed now. Great!

To get started you need Poetry's bin directory (/root/.local/bin) in your `PATH`
environment variable.

Add `export PATH="/root/.local/bin:$PATH"` to your shell configuration file.

Alternatively, you can call Poetry explicitly with `/root/.local/bin/poetry`.

You can test that everything is set up by executing:

`poetry --version`



In [2]:
# add to path
import os
os.environ["PATH"] = '/root/.local/bin' + os.pathsep + os.environ["PATH"]
!which poetry

/root/.local/bin/poetry


In [6]:
!mkdir art
%cd art
!poetry init --no-interaction
!poetry config virtualenvs.create false
!poetry add git+https://github.com/Trusted-AI/adversarial-robustness-toolbox.git
%cd ..

/content/art/art
Skipping virtualenv creation, as specified in config file.

Updating dependencies
Resolving dependencies... (0.6s)

Package operations: 1 install, 3 updates, 0 removals

  - Updating numpy (1.26.4 -> 2.2.3): Pending...
  - Updating numpy (1.26.4 -> 2.2.3): Downloading... 0%
  - Updating numpy (1.26.4 -> 2.2.3): Downloading... 90%
  - Updating numpy (1.26.4 -> 2.2.3): Downloading... 100%
  - Updating numpy (1.26.4 -> 2.2.3): Installing...
  - Updating numpy (1.26.4 -> 2.2.3)
  - Updating scipy (1.13.1 -> 1.15.2): Pending...
  - Updating scipy (1.13.1 -> 1.15.2): Downloading... 0%
  - Updating scipy (1.13.1 -> 1.15.2): Downloading... 40%
  - Updating scipy (1.13.1 -> 1.15.2): Downloading... 80%
  - Updating scipy (1.13.1 -> 1.15.2): Downloading... 100%
  - Updating scipy (1.13.1 -> 1.15.2): Installing...
  - Updating scipy (1.13.1 -> 1.15.2)
  - Updating setuptools (75.1.0 -> 76.0.0): Pending...
  - Updating setuptools (75.1.0 -> 76.0.0): Installing...
  - Updating setup

# Running database reconstruction attacks on the Iris dataset

In this tutorial we will show how to run a database reconstruction attack on the Iris dataset and evaluate its effectiveness against models trained non-privately (i.e., naively with scikit-learn) and models trained with differential privacy guarantees.

## Preliminaries

The database reconstruction attack takes a trained machine learning model `model`, which has been trained by a training dataset of `n` examples.  Then, using `n-1` examples of the training dataset (i.e., with the target row removed), we seek to reconstruct the `n`th example of the dataset by using `model`.

In this example, we train a Gaussian Naive Bayes classifier (`model`) with the training dataset, then remove a single row from that dataset, and seek to reconstruct that row using `model`. For typical examples, this attack is successful up to machine precision.

We then show that launching the same attack on a ML model trained with differential privacy guarantees provides protection for the training dataset, and prevents learning the target row with precision.

## Example usage

## Load data

First, we load the data of interest and split into train/test subsets.

In [7]:
%cd art
!poetry add scikit-learn==1.1.0 diffprivlib
%cd ..

/content/art/art
Skipping virtualenv creation, as specified in config file.
Using version ^0.6.5 for diffprivlib

Updating dependencies
Resolving dependencies... (0.6s)

Package operations: 1 install, 1 update, 0 removals

  - Downgrading scikit-learn (1.6.1 -> 1.1.0): Pending...
  - Downgrading scikit-learn (1.6.1 -> 1.1.0): Downloading... 0%
  - Downgrading scikit-learn (1.6.1 -> 1.1.0): Downloading... 100%
  - Downgrading scikit-learn (1.6.1 -> 1.1.0): Preparing...
  - Downgrading scikit-learn (1.6.1 -> 1.1.0): Failed

PEP517 build of a dependency failed

Backend subprocess exited when trying to invoke build_wheel

    | Command '['/tmp/tmpavx05rib/.venv/bin/python', '/root/.local/share/pypoetry/venv/lib/python3.11/site-packages/pyproject_hooks/_in_process/_in_process.py', 'build_wheel', '/tmp/tmpipi8i_52']' returned non-zero exit status 1.
    | 
    | Partial import of sklearn during the build process.
    | setup.py:128: DeprecationWarning: 
    | 
    |   `numpy.distutils` is de

In [8]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

dataset = datasets.load_iris()

In [9]:
x_train, x_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.2)

## Train model

We can now train a Gaussian naive Bayes classifier using the full training dataset. This is the model that will be used to attack the training dataset later.

In [10]:
import sklearn.naive_bayes as naive_bayes
from art.estimators.classification.scikitlearn import ScikitlearnGaussianNB

model1 = naive_bayes.GaussianNB().fit(x_train, y_train)
non_private_art = ScikitlearnGaussianNB(model1)

In [11]:
print("Model accuracy (on the test dataset): {}".format(model1.score(x_test, y_test)))

Model accuracy (on the test dataset): 0.9666666666666667


## Launch and evaluate attack

We now select a row from the training dataset that we will remove. This is the **target row** which the attack will seek to reconstruct. The attacker will have access to `x_public` and `y_public`.

In [12]:
target_row = int(np.random.random() * x_train.shape[0])

x_public = np.delete(x_train, target_row, axis=0)
y_public = np.delete(y_train, target_row, axis=0)

We can now launch the attack, and seek to infer the value of the target row. This is typically completed in less than a second.

In [13]:
from art.attacks.inference.reconstruction import DatabaseReconstruction

dbrecon = DatabaseReconstruction(non_private_art)

x, y = dbrecon.reconstruct(x_public, y_public)

We can evaluate the accuracy of the attack using root-mean-square error (RMSE), showing a high level of accuracy in the inferred value.

In [14]:
print("Inference RMSE: {}".format(
    np.sqrt(((x_train[target_row] - x) ** 2).sum() / x_train.shape[1])))

Inference RMSE: 9.357607689679968e-09


We can confirm that the attack also inferred the correct label `y`.

In [15]:
np.argmax(y) == y_train[target_row]

True

# Attacking a model trained with differential privacy

We can mitigate against this attack by training the public ML model with differential privacy.  We will use [diffprivlib](https://github.com/Trusted-AI/differential-privacy-library) to train a differentially private Gaussian naive Bayes classifier. We can mitigate against any loss in accuracy of the model by choosing an `epsilon` value appropriate to our needs.

## Train the model

In [17]:
%cd art/
!poetry add diffprivlib
%cd ..

/content/art/art
Skipping virtualenv creation, as specified in config file.
Using version ^0.6.5 for diffprivlib

Updating dependencies
Resolving dependencies... (0.1s)

Package operations: 1 install, 0 updates, 0 removals

  - Installing diffprivlib (0.6.5): Pending...
  - Installing diffprivlib (0.6.5): Downloading... 0%
  - Installing diffprivlib (0.6.5): Downloading... 100%
  - Installing diffprivlib (0.6.5): Installing...
Installing /usr/local/lib/python3.11/dist-packages/tests/__init__.py over existing file
  - Installing diffprivlib (0.6.5)

Writing lock file
/content/art


In [18]:
from diffprivlib import models

model2 = models.GaussianNB(bounds=([4.3, 2.0, 1.1, 0.1], [7.9, 4.4, 6.9, 2.5]), epsilon=3).fit(x_train, y_train)
private_art = ScikitlearnGaussianNB(model2)

model2.score(x_test, y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


0.9333333333333333

## Launch and evaluate attack

We then launch the same attack as before. In this case, the attack may take a number of seconds to return a result.

In [19]:
dbrecon = DatabaseReconstruction(private_art)

x_dp, y_dp = dbrecon.reconstruct(x_public, y_public)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator.

In this case, the RMSE shows our attack has not been as successful

In [20]:
print("Inference RMSE (with differential privacy): {}".format(
    np.sqrt(((x_train[target_row] - x_dp) ** 2).sum() / x_train.shape[1])))

Inference RMSE (with differential privacy): 1.9811610832886488


This is confirmed by inspecting the inferred value and the true value.

In [21]:
x_dp, x_train[target_row]

(array([[4.80000026, 2.99999955, 1.40000055, 0.09999968]]),
 array([5.9, 3.2, 4.8, 1.8]))

In fact, the attack may not even be able to correctly infer the target label.

In [22]:
np.argmax(y_dp), y_train[target_row]

(0, 1)